# Preprocessing the .txt data file, and save to excel file

- For AISG-SmartRx project (A*STAR IHPC, SGH)
- By Yang Feng (IHPC) and team, 14-Apr-2023
- the code was tested in Python 3.9.13 (anaconda)

In [1]:
import numpy as np
import pandas as pd
import time

from platform import python_version
print('Current Python version: {0}'.format(python_version()))
import sys
print('Current Envronment: {0}'.format(sys.executable))

# conda env list

Current Python version: 3.9.13
Current Envronment: C:\Users\yangf\Anaconda3\python.exe


# Define the path of raw .txt data file

In [2]:
# read a txt file
# --------------------------
# if txt file in a different folder, e.g. "Data (SmartRx)"
# --------------------------
# folder = "Data (SmartRx)"
# file_name = "Sample extracted data_deidentified 2.txt"
# file_full_ = folder+'/'+ file_name

# --------------------------
# if txt file in the same folder
# --------------------------
file_full_ = 'Sample extracted data_deidentified 2 - Copy.txt'
print(file_full_)

Sample extracted data_deidentified 2 - Copy.txt


# start pre-processing

In [3]:
# ------------------------------------------------
# define the functions
# ------------------------------------------------

def is_nric(val_str_):
    """to check whether a string is a NRIC"""
    n_numeric_ = 2
    if len(val_str_) < n_numeric_+1: # assume at least one letter + n_numeric_ numbers in the dataset for a NRIC
        return False 
    flag1_ = val_str_[0] in {'S', 'G', 'F', 'T'}  # NRIC should start with one of thoese capital letters
    if flag1_: 
        try:
            int(val_str_[1:n_numeric_+1])
            return True
        except:
            return False
    else: 
        return False

    
def convert_txt_to_df(file_full_):
    """preprocessing to covert all records into a dataframe"""
    one_record_ = ''
    record_no_ = 0
    with open(file_full_) as f:
        line = f.readline()
        line_no_ = 1
        while line:
#             print('-------------------------------------')
#             print('-------Line %d (length=%d):' %(line_no_, len(line)))
            # -----------------------------------
            nric_flag_ = is_nric(line.strip())  # check whether the current line starts with a NRIC
            if nric_flag_:                
#                 print(one_record_)
                # -----------------------------
                # convert to list 
                dat_ = one_record_.split("^|^")  # convert a complete record into a list
    #             print('\n&&&&& No. of cells in current line %d:\n, %s' %(len(dat_), dat_))

                if record_no_ == 0:             
                    df_records_ = pd.DataFrame(columns=dat_)
                    record_no_ +=1            
                else:
                    df_records_.loc[len(df_records_)] = dat_  # append a list as one row into dataframe
                    record_no_ +=1
                # ---------------------------
                one_record_ = ''  # initialize for a new record

            if len(line.strip()) > 0 & ~nric_flag_:  # not an empty line
                one_record_= one_record_.strip() + ' ' + line.strip()  # remove empty

            # -------------------------------
            # read a new line from the text data
            line = f.readline()
            line_no_+=1
        # --------------------------------------
        # process the last record in the txt file
#         print(one_record_)  # print the last record
        dat_ = one_record_.split("^|^")
        df_records_.loc[len(df_records_)] = dat_
    return df_records_, line_no_

## call the function to process 

In [4]:
# --------------------------------
# call the function to process the text file and save all records into a excel file
# --------------------------------
tic_ = time.time()  # start time
df_records_,line_no_ = convert_txt_to_df(file_full_)
time_est_ = time.time()-tic_
print('************************************************************')
print('------------------------------------------------------------')
print('Text data processing completed in %s seconds' %time_est_)
print('Text file inlcudes %d lines, converted to %d records' %(line_no_, df_records_.shape[0]))
print('------------------------------------------------------------')
# --------------------------------
display(df_records_)   # display the dataframe (contianing all records)

# --------------------------------
# save to a excel file
# --------------------------------
file_output_ = 'output_data_.xlsx'
df_records_.to_excel(file_output_, index=False)
print('************************************************************')
print('All %d records are saved to %s' %(df_records_.shape[0], file_output_))

************************************************************
------------------------------------------------------------
Text data processing completed in 0.023919105529785156 seconds
Text file inlcudes 41 lines, converted to 10 records
------------------------------------------------------------


,Patient_NRIC,Visit_No,Order_Review_Category,Order_ID,Order_Name,Route,DosageLow,DosageHigh,Frequency,Duration,...,Order_Requested,Order_Stop_Date,Function_Code_Num,Order_Status_History_Created_When,Order_Status_History_Created_By,Order_Status_Code,Order_Status_Description,Reason,TaskName,ScheduledDtm
0,S101*****,12000******,Medication (Non-Parenteral),001GKV2VC,Amlodipine Tablet,PO,5,,OM,30,...,2018-03-05 11:47:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:50.487,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Amlodipine Tablet PO 5 mg, OM",2018-03-06 08:00:00.000
1,S101*****,12000******,Medication (Non-Parenteral),001GKV2VH,GliPIzide Tablet,PO,2.5,,Pre-dinner,30,...,2018-03-05 11:48:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:50.550,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","GliPIzide Tablet PO 2.5 mg, Pre-dinner reduced...",2018-03-07 17:30:00.000
2,S101*****,12000******,Medication (Non-Parenteral),001GKV2VS,Sennosides 7.5mg Tablet,PO,1,,ON,30,...,2018-03-05 11:48:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:50.657,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Sennosides 7.5mg Tablet PO 1 tablet, ON PRN Co...",2018-03-08 20:00:00.000
3,S101*****,12000******,Medication (Non-Parenteral),001GKV2WK,Aspirin Tablet,PO,100,,OM,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:50.940,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Aspirin Tablet PO 100 mg, OM Take with or aft...",2018-03-12 08:00:00.000
4,S101*****,12000******,Medication (Non-Parenteral),001GKV2WN,"Neurobion Tablet [Vit B1 100mg, B6 200mg, B12...",PO,1,,OM,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:50.987,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Neurobion Tablet [Vit B1 100mg, B6 200mg, B12...",2018-03-06 08:00:00.000
5,S101*****,12000******,Medication (Non-Parenteral),001GKV2WT,Calcium Acetate 667mg Tablet,PO,1,,TDS,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:51.017,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Calcium Acetate 667mg Tablet PO 1 tablet, TDS ...",2018-03-06 17:30:00.000
6,S101*****,12000******,Medication (Non-Parenteral),001GKV2X2,Ketoprofen 30mg Plaster,Trans-Dermal,1,,BD,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:51.127,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Ketoprofen 30mg Plaster Trans-Dermal 1 patch, ...",2018-03-06 08:00:00.000
7,S101*****,12000******,Medication (Non-Parenteral),001GKV2X9,Famotidine Tablet,PO,20,,OM,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:51.173,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...","Famotidine Tablet PO 20 mg, OM",2018-03-07 08:00:00.000
8,S101*****,12000******,Medication (Non-Parenteral),001GKV2XM,Sangobion [Ferrous Gluconate Compound] Capsule,PO,2,,OM,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:51.283,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...",Sangobion [Ferrous Gluconate Compound] Capsule...,2018-03-07 08:00:00.000
9,S101*****,12000******,Medication (Non-Parenteral),001GKV2XQ,Hypromellose 0.3% Eye Drop,Ophthalmic,1,,QDS,30,...,2018-03-05 11:49:00.000,2018-03-24 12:00:00.000,5,2018-03-05 11:50:51.313,Doctor Name,AUA1,Ordered,"Electronically Signed by Doctor Name, (Doctor)...",Hypromellose 0.3% Eye Drop Ophthalmic 1 drop t...,2018-03-06 02:00:00.000


************************************************************
All 10 records are saved to output_data_.xlsx
